# РАСЧЕТ ПЛАНА ПРОИЗВОДСТВА НА 2025 г.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Загрузка исходных данных для расчета

In [10]:
actual_items = pd.read_excel('data/actual_items.xlsx')
current_stocks = pd.read_excel('data/stocks_2024.xlsx') # Остатки на 31.12.2024
current_stocks['item'] = current_stocks['item'].str.replace(', шт', '')
current_stocks.fillna(0, inplace=True)
current_stocks['stock_level'] = current_stocks['stock_level'].astype('int')
sales_plan = pd.read_excel('data/sales_plan_2025.xlsx')

In [11]:
month_mapping = {'Январь': 1, 'Февраль': 2, 'Март': 3, 'Апрель': 4, 'Май': 5, 'Июнь': 6, 'Июль': 7,
                            'Август': 8, 'Сентябрь': 9, 'Октябрь': 10, 'Ноябрь': 11, 'Декабрь': 12, }
month = sales_plan['date'].str.replace(' г.', "").str.split(' ', expand=True)[0].map(month_mapping)
year = sales_plan['date'].str.replace(' г.', "").str.split(' ', expand=True)[1]
temp = month.to_frame().join(year)
temp[0] = temp[0].astype('str')
temp[1] = temp[1].astype('str')
temp['date'] = temp[0] + '-' + temp[1]
temp['date'] = pd.to_datetime(temp['date'], format="%m-%Y")
sales_plan['date'] = temp['date']

## Расчет плана производства

Функции для расчета:

In [4]:
def calculate_production(actual_items, stocks, sales_plan, safety_level=41, working_days=20, start_date="2024-10-01"):
    production_plan = {}
    actual_items = actual_items['item'].to_list()
    sales_plan = sales_plan.query('date == @start_date')
    for x in actual_items:
        if x in stocks['item'].to_list():
            stock_level = stocks.query('item == @x')['stock_level'].tolist()[0]
        else: 
            stock_level = 0
        if x in sales_plan['item'].to_list():
            sales_quantity = sales_plan.query('item == @x')['quantity'].tolist()[0]
        else:
            sales_quantity = 1
        sales_per_day = sales_quantity / working_days
        if (stock_level / sales_per_day - safety_level) >0:
            production_plan[x] = 0
        else:
            production_plan[x] = round((safety_level - stock_level / sales_per_day) * sales_per_day, -3)
    return production_plan

In [6]:
def calculate_stocks(actual_items, stocks, sales_plan, production_plan, start_date="2024-10-01"):
    sales_plan = sales_plan.query('date == @start_date')
    df = (actual_items
          .merge(stocks, how='left', on='item')
          .merge(sales_plan, how='left', on='item')
          .merge(production_plan, how='left', on='item'))
    df['stock_level'] = df['stock_level'] + df['production_volume'] - df['quantity']
    return df[['item', 'stock_level']]

Расчет плана производства:

In [123]:
planning_period = ['2025-01-01', '2025-02-01', '2025-03-01', '2025-04-01', '2025-05-01', 
                   '2025-06-01', '2025-07-01', '2025-08-01', '2025-09-01', '2025-10-01', '2025-11-01', '2025-12-01']
safety_level = [54, 52, 49, 66, 74, 62, 75, 65, 47, 52, 49, 46]
production_plan = {}
stocks = current_stocks
planning_stocks = []
for period, level in zip(planning_period, safety_level):
    plan = calculate_production(actual_items, stocks, sales_plan, safety_level=level, start_date=period)
    production_plan[period] = plan
    plan = pd.DataFrame.from_dict([plan]).T.reset_index().rename(columns={'index':'item', 0:'production_volume'})
    new_stocks = calculate_stocks(actual_items, stocks, sales_plan, plan, start_date=period)
    planning_stocks.append(new_stocks)
    stocks = new_stocks

## ИТОГОВЫЙ ПЛАН ПРОИЗВОДСТВА

In [124]:
final_production_plan = pd.DataFrame(production_plan).reset_index().rename(columns={'index':'item'})
final_production_plan.head()

,item,2025-01-01,2025-02-01,2025-03-01,2025-04-01,2025-05-01,2025-06-01,2025-07-01,2025-08-01,2025-09-01,2025-10-01,2025-11-01,2025-12-01
0,Аир корневища 75г,4000.0,7000.0,8000.0,4000.0,4000.0,6000.0,4000.0,5000.0,8000.0,7000.0,8000.0,9000.0
1,Алтей корни 75г,0.0,0.0,0.0,0.0,1000.0,2000.0,1000.0,2000.0,3000.0,2000.0,3000.0,3000.0
2,Багульник болотный побеги 50г,5000.0,8000.0,8000.0,5000.0,5000.0,6000.0,5000.0,6000.0,9000.0,8000.0,9000.0,10000.0
3,Береза почки 50г,5000.0,9000.0,9000.0,5000.0,5000.0,6000.0,5000.0,6000.0,10000.0,8000.0,10000.0,10000.0
4,Бессмертник песчаный цветки 30г,18000.0,16000.0,17000.0,10000.0,10000.0,13000.0,9000.0,12000.0,19000.0,16000.0,17000.0,21000.0


In [125]:
sales_by_month = sales_plan.groupby(by='date').sum().quantity.reset_index()
production_plan_by_month = final_production_plan.sum().drop('item',axis=0).reset_index().rename(columns={'index':'date', 0:'production_volume'})
production_plan_by_month['date'] = pd.to_datetime(production_plan_by_month['date'])
production_plan_by_month.merge(sales_by_month, how='left', on='date')

,date,production_volume,quantity
0,2025-01-01,2476000.0,2551382
1,2025-02-01,2887000.0,2827439
2,2025-03-01,3097000.0,3127585
3,2025-04-01,1903000.0,1972991
4,2025-05-01,1767000.0,1717764
5,2025-06-01,2223000.0,2210357
6,2025-07-01,1713000.0,1706510
7,2025-08-01,2137000.0,2111786
8,2025-09-01,3378000.0,3471795
9,2025-10-01,2874000.0,2923500


## ИТОГОВЫЙ ЭКСПОРТ В EXCEL

In [126]:
final_production_plan.to_excel('data/final_production_plan.xlsx')